In [3]:
!pip install python-docx
#!pip install textract
!pip install PyMuPDF
#!pip install pywin32

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 6.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 76.8 MB/s eta 0:00:00:00:010:01
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 22.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 52.0 MB/s eta 0:00:00:00:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
#!pip install PyPDF2
from transformers import AutoTokenizer
from tokenizers.normalizers import NFC
#import fitz
import re
import os
#from docx import Document
#import textract
import pandas as pd
import zipfile

In [54]:
txt='<Description>{"namn": "Mona Mårtensson", "ålder": 55, "kön": "kvinna", "bakgrund": "Projektledare på reklambyrå. Aktiv inom yoga och löpning.", "medicinsk_historik": "God generell hälsa. Mild astma behandlad med inhalator. Modern hade typ 2 Diabetes.", "medicinering": ["Inhalator med budesonid/formoterol vid behov"], "allergier": "Pollensallergi med säsongbunden rinit"}\n\nAktuellt:\nPatient söker för plötslig synstörning på höger öga. Anamnes indikerar högt stresspåslag i arbetet den senaste månaden. Fundoskopi avslöjar en retinal venocklusion. \n\nPlanerar utredning och behandling inom oftalmologi. Stresshantering rekommenderas.\n\nAstman är välkontrollerad och inhalatorn används endast vid behov.\n</Description>\n<Summary>\n*Sjukdomshistoria*:\nMild astma, användning av inhalator vid behov. Modern hade typ 2 Diabetes.\n\n*Sökorsaker*:\nPlötslig synstörning på höger öga, retinal venocklusion konstaterad.\n\n*Åtgärder*:\nOftalmologi utredning/behandling, stressreducering.\n</Summary>'

match = re.split(r"Sjukdomshistoria", txt,maxsplit=1)
print(match[0])

<Description>{"namn": "Mona Mårtensson", "ålder": 55, "kön": "kvinna", "bakgrund": "Projektledare på reklambyrå. Aktiv inom yoga och löpning.", "medicinsk_historik": "God generell hälsa. Mild astma behandlad med inhalator. Modern hade typ 2 Diabetes.", "medicinering": ["Inhalator med budesonid/formoterol vid behov"], "allergier": "Pollensallergi med säsongbunden rinit"}

Aktuellt:
Patient söker för plötslig synstörning på höger öga. Anamnes indikerar högt stresspåslag i arbetet den senaste månaden. Fundoskopi avslöjar en retinal venocklusion. 

Planerar utredning och behandling inom oftalmologi. Stresshantering rekommenderas.

Astman är välkontrollerad och inhalatorn används endast vid behov.
</Description>
<Summary>
*


In [38]:
pre_path = '../../data/finetuning/hus75/Pre-training'
directory = os.fsencode('../../data/finetuning/hus75/Pre-training')
reg = r"(\n \n )"
data = list()
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".doc"):
        print(filename)
        text = textract.process(pre_path + '/'+ filename,language='swe')
        text = text.decode("utf-8")
        text = re.sub(reg,' ',text)
        data.append(text)
    elif filename.endswith(".docx"):
        doc = Document(pre_path + '/'+ filename)
        complete_text = ''
        for p in doc.paragraphs:
            text = p.text
            text = re.sub(reg,' ',text)
            complete_text+=text
        data.append(complete_text)
    elif filename.endswith(".pdf") or filename.endswith(".py"):
        doc = fitz.open(pre_path + '/'+ filename)
        #reader = PdfReader(pre_path + '/'+ filename)
        #num_pages = len(reader.pages)
        complete_text = ''
        for page in doc:
            text = page.get_text()
            text = re.sub(reg,' ',text)
            complete_text+=text
        data.append(complete_text)
data = pd.DataFrame(data,columns = ['text'])
data.to_json('pre_train_data.json',compression = None)
print('done')

DFM3M1ALEL.doc
DSM2-2 Kemisk analys.doc
DSM2-1 Infektionsimmunologi.doc
DSM2-2 Statusfynd.doc
IDT munta.doc
DSM2-2 EKG.doc
DSMfarmakologiTGKJ.doc
done


In [41]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-13b-chat-hf',token = '')


In [46]:
print(tokenizer.backend_tokenizer.normalizer.normalize_str('Hejsan jag heter Kalle'))

▁Hejsan▁jag▁heter▁Kalle


In [11]:
pre_path = '../../data/finetuning/hus75/apkg'
directory = os.fsencode(pre_path)
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".apkg"):
        with zipfile.ZipFile(pre_path + '/'+ filename, 'r') as zip_ref:
            zip_ref.extractall('../../data/finetuning/hus75/unzipped/'+filename)
        print(filename)

BV1.apkg
BV2.apkg
Klinisk Medicin_ Neuro, Sinnen, Psyke (T9).apkg
LÄK BasV2_M1–3.apkg
Ultraljud BV3.apkg
BV3- Alland Koyikoyan.apkg
BV3.apkg
Basvetenskap 5 (Filip Borg).apkg
DFM2.apkg
Basvetenskap 4 (Filip Borg).apkg
Klinisk Medicin DX1-6 + VetU av HA & Co.apkg
Medicin__Basvetenskap 6.apkg
Histologi (Filip Borg).apkg
Anatomi (Filip Borg).apkg
Klinisk medicin 1.apkg
DFM3__Moment 1.apkg
BV5 - Aurora Sandart.apkg
T5. MVM.apkg
MDBI moment 2.apkg
Medicin__Basvetenskap 5.2.apkg
T1, Alland Koyikoyan.apkg
T7__Klinisk kirurgi__Tentor VT_HT 2014-2021.apkg
T6__Hud-kursen Tentasamling.apkg
T6__Infektionskursen Tentor Kortsvar.apkg
T6__Tentasamling Klin Farm & Geriatrik.apkg
LÄK BasV1.apkg
Medicin__Basvetenskap 4.apkg
DFM3__Moment 2.apkg
T5. Klinisk kemi MCQ.apkg
MDBI moment 1.apkg
Klinisk Medicin__Kirurgi.apkg
DFM1 tenta.apkg
MDBI.apkg
Bv4 3D Slicer.apkg
DFM2 tenta.apkg
DSM2.2 Medicinsk diagnostik Gamla tentor Victoria Nord.apkg
Medicin__Basvetenskap 5.apkg
Klinisk medicin 1 (T5).apkg
T6__Tentor

In [15]:
df = pd.read_csv('../Finetuning/flashcards/test.csv')

In [24]:
df.iloc[20031]['Question'] == "     "

True